In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Data Profiling Setup Complete")

Data Profiling Setup Complete


In [11]:
toilet_locations = "../data/ext_data/data/public_restrooms_20251016.csv"
df_toilet_raw = pd.read_csv(toilet_locations)
print(df_toilet_raw.head())
print(len(df_toilet_raw))

            Facility Name Location Type     Operator                   Status  \
0     Battery Park Zone 1          Park    NYC Parks          Not Operational   
1   Carl Schurz Main Lawn          Park    NYC Parks          Not Operational   
2             Tappen Park          Park    NYC Parks          Not Operational   
3       Refinery Building          Park  Domino Park              Operational   
4  Gravesend Library, BPL       Library          BPL  Closed for Construction   

         Open Hours of Operation         Accessibility  \
0         NaN                NaN                   NaN   
1         NaN                NaN  Partially Accessible   
2         NaN                NaN                   NaN   
3  Year Round           6am-11pm                   NaN   
4  Year Round                NaN      Fully Accessible   

                         Restroom Type        Changing Stations  \
0                                  NaN                      Yes   
1            Multi-Stall W/M R

In [12]:
df_toilet_raw.groupby("Status").count() / len(df_raw)

,Facility Name,Location Type,Operator,Open,Hours of Operation,Accessibility,Restroom Type,Changing Stations,Additional Notes,Website,Latitude,Longitude,Location
Status,,,,,,,,,,,,,
Closed,0.000938,0.000938,0.000938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000938,0.000938,0.000938,0.000938
Closed for Construction,0.015947,0.015947,0.015947,0.015947,0.004690,0.008443,0.009381,0.009381,0.001876,0.015009,0.015947,0.015947,0.015947
Not Operational,0.068480,0.068480,0.068480,0.000938,0.002814,0.054409,0.052533,0.060038,0.000000,0.000000,0.068480,0.068480,0.068480
Operational,0.914634,0.914634,0.914634,0.914634,0.905253,0.772983,0.803002,0.848968,0.064728,0.197936,0.914634,0.914634,0.914634


Tidbit:
* 91% operational
* 9% are non-operational or closed for construction


Plan
* Join the public restrooms database with the the park information, and then use the coordinates to do a FIPS association

In [15]:
print(f"Toilet Inventory Columns: {df_toilet_raw.columns}")

park_master_locations = "../data/ext_data/data/parks_properties_20250930.csv"
df_parks = pd.read_csv(park_master_locations)
print(f"Park Metadata Columns: {df_parks.columns}")

Toilet Inventory Columns: Index(['Facility Name', 'Location Type', 'Operator', 'Status', 'Open',
       'Hours of Operation', 'Accessibility', 'Restroom Type',
       'Changing Stations', 'Additional Notes', 'Website', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')
Park Metadata Columns: Index(['ACQUISITIONDATE', 'ACRES', 'ADDRESS', 'BOROUGH', 'CLASS',
       'COMMUNITYBOARD', 'COUNCILDISTRICT', 'DEPARTMENT', 'EAPPLY', 'GISOBJID',
       'GISPROPNUM', 'GlobalID', 'JURISDICTION', 'LOCATION', 'MAPPED',
       'NAME311', 'NYS_ASSEMBLY', 'NYS_SENATE', 'OBJECTID', 'OMPPROPID',
       'PARENTID', 'PERMIT', 'PERMITDISTRICT', 'PERMITPARENT', 'PIP_RATABLE',
       'PRECINCT', 'RETIRED', 'SIGNNAME', 'SUBCATEGORY', 'TYPECATEGORY',
       'US_CONGRESS', 'WATERFRONT', 'ZIPCODE', 'multipolygon'],
      dtype='object')


In [26]:
park_sites_location = "../data/ext_data/data/parks_inspection_program_–_all_sites_mapped_20251018.csv"
park_inspections_location = "../data/ext_data/data/parks_inspection_program_–_inspections_20251018.csv"
park_restroom_inspections_location = "../data/ext_data/data/parks_inspection_program_–_public_restroom_inspections_20251018.csv"
park_restroom_rooms_inspections_location = "../data/ext_data/data/parks_inspection_program_–_public_restroom_individual_room_inspections_20251018.csv"

park_sites_df = pd.read_csv(park_sites_location)
park_inspections_df = pd.read_csv(park_inspections_location)
park_restroom_inspections_df = pd.read_csv(park_restroom_inspections_location)
park_restroom_rooms_inspections_df = pd.read_csv(park_restroom_rooms_inspections_location)

print(f"Park Sites Columns: {park_sites_df.columns}")
print(f"Inspection Columns: {park_inspections_df.columns}")
print(f"Restroom Inspection Columns: {park_restroom_inspections_df.columns}")
print(f"Restroom Rooms Inspection Columns: {park_restroom_rooms_inspections_df.columns}")

Park Sites Columns: Index(['PropNum', 'Prop ID', 'Boro', 'AMPSDistrict', 'Prop Name', 'Site Name',
       'Prop Location', 'Site Location', 'Acres', 'Category', 'Sub-Category',
       'Rated', 'Reason Not Rated', 'Council District', 'ZipCode',
       'COMMUNITYBOARD', 'Jurisdiction', 'NYSAssembly', 'NYSSenate',
       'USCongress', 'Precinct', 'PublicRestroom', 'Multipolygon', 'Sourcefc'],
      dtype='object')
Inspection Columns: Index(['Prop ID', 'AMPSDistrict', 'Inspection ID', 'Season', 'Round', 'Date',
       'BeginInspection', 'EndInspection', 'Inspection Year', 'inspector',
       'inspector2', 'Overall Condition', 'Cleanliness', 'Safety Condition',
       'Structural Condition', 'VisitorCount', 'Closed?', 'Comments',
       'InspectionType', 'inspAddedDate'],
      dtype='object')
Restroom Inspection Columns: Index(['InspectionID', 'PRNumber', 'PR Overall Condition', 'PR Litter',
       'PR Graffiti', 'PR Amenities', 'PR Structural'],
      dtype='object')
Restroom Rooms Inspec

Based on https://docs.google.com/spreadsheets/d/17LSfFUkhHPHJS4hszOYMVmVyU3xVF7J_dILhtIwWKKA/edit?gid=2034137855#gid=2034137855, the join pattern is:
* Each INSPECTION record will have 0..N restroom inspections and 0..N restroom room inspections
* Each INSPECTION record has a Prop ID
* In the INSPECTION record, there is a Prop ID that can be joined to the Park Sites table

In [ ]:
from shapely.wkt import loads
park_sites_df['geometry'] = park_sites_df['Multipolygon'].apply(loads)
park_sites_df['park_lat'] = park_sites_df['geometry'].apply(lambda x:x.centroid.y)
park_sites_df['park_lng'] = park_sites_df['geometry'].apply(lambda x:x.centroid.x)

# Got lat/long for park centers, then use FIPS lookup to associate with fips
park_sites_df[park_sites_df["Prop ID"] == "B288"]

,PropNum,Prop ID,Boro,AMPSDistrict,Prop Name,Site Name,Prop Location,Site Location,Acres,Category,...,NYSAssembly,NYSSenate,USCongress,Precinct,PublicRestroom,Multipolygon,Sourcefc,geometry,park_lat,park_lng
1424,B288,B288,B,07,Slope Park Playground,Slope Park Playground,6 Ave. between 18 St. and 19 St.,6 Ave. between 18 St. and 19 St.,0.504,Small Park,...,51,20,10,72,Yes,MULTIPOLYGON (((-73.98913202082416 40.66139567...,Property,MULTIPOLYGON (((-73.98913202082416 40.66139567...,40.661269,-73.989511


In [ ]:
import requests
from tqdm.notebook import tqdm

# This does not work because of the government shutdown...
def get_fips_from_coords(lat, lng):
    url = f"https://geocoding.census.gov/geocoder/geographies/coordinates"
    params = {
        'x': lng,
        'y': lat,
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current',
        'format': 'json'
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data['result']['geographies']:
        # Returns state, county, tract, block group FIPS
        return data['result']['geographies']['Census Blocks'][0]
    return None

print(f"Attempting to FIPS identify {len(park_sites_df)} records...")

def extract_fips_info(fips_dict):
    if fips_dict is None:
        return None
    return fips_dict.get('GEOID', None)

tqdm.pandas()

sample_record = park_sites_df[park_sites_df["Prop ID"] == "B288"]
print(get_fips_from_coords(sample_record.park_lat, sample_record.park_lng))


#park_sites_df['fips_info'] = park_sites_df.progress_apply(
#    lambda row: get_fips_from_coords(row['park_lat'], row['park_lng']), axis=1
#)
#park_sites_df['fips'] = park_sites_df['fips_info'].apply(extract_fips_info)

Attempting to FIPS identify 6385 records...


ConnectTimeout: HTTPSConnectionPool(host='geocoding.census.gov', port=443): Max retries exceeded with url: /geocoder/geographies/coordinates?x=-73.98951078309588&y=40.661268791863826&benchmark=Public_AR_Current&vintage=Current_Current&format=json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1097291d0>, 'Connection to geocoding.census.gov timed out. (connect timeout=None)'))

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

  # Download Census TIGER shapefiles
geotract_file = "../data/ext_data/tl_2025_36_tract/tl_2025_36_tract.shp"
counties = gpd.read_file(geotract_file)

sample_record = park_sites_df[park_sites_df["Prop ID"] == "B288"]
point = Point(sample_record.park_lng, sample_record.park_lat)
fips = counties[counties.contains(point)]['GEOID'].iloc[0]

DataSourceError: ../data/ext_data/data/tl_2025_36_tract/tl_2025_36_tract.shp: No such file or directory